In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [7]:
def getFinancialInfo(symbol):
    url="https://finance.yahoo.com/quote/"+symbol+"?p="+symbol
    r = requests.get(url)
    soup = bs(r.content)
    trs= soup.find_all('tr')
    finalTag= '1y Target Est'
    namVal = {}
    names = []
    values = []
    for i in range(len(trs)):
        for j in range(len(trs[i].contents)):
            if j==0:
                try:
                    name = trs[i].contents[j].text
                    names.append(name)
                except:
                    continue
            if j==1:
                try:
                    value = trs[i].contents[j].text
                    values.append(value)
                except:
                    continue
        #namVal[name]=value
        #if name == finalTag:
           # break
    return names,values       

         

def getCompanyList():
    url1="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    r = requests.get(url1)
    soup = bs(r.content)
    tbody= soup.find_all('tbody')
    tickerSymbols = []

    for i in range(len(tbody[0].contents)):
        if i<2:
            continue
        if i%2 != 0:
            continue
            
        symbol = tbody[0].contents[i].contents[1].text
        tickerSymbols.append(symbol.strip("\n"))
        if len(tickerSymbols)==505:
            break
    
    return tickerSymbols



data = {"symbol":[],
       "metric":[],
       "value":[]}

tickerSymbols = getCompanyList()

for symbol in tickerSymbols:
    names, values = getFinancialInfo(symbol)
    for i in range(len(names)):
        data["symbol"].append(symbol)
        data["metric"].append(names[i])
        data["value"].append(values[i])
        
df= pd.DataFrame(data)
df.to_csv("financialData.csv")

In [14]:
df.head()

,symbol,metric,value
0,MMM,Previous Close,181.54
1,MMM,Open,181.37
2,MMM,Bid,180.97 x 1100
3,MMM,Ask,181.01 x 800
4,MMM,Day's Range,180.45 - 182.57


In [31]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["metric"], inplace=True)

In [36]:
df.head()

,symbol,metric,value
0,MMM,Previous Close,181.54
1,MMM,Open,181.37
2,MMM,Bid,180.97 x 1100
3,MMM,Ask,181.01 x 800
4,MMM,Day's Range,180.45 - 182.57


In [37]:
df.to_csv("finalFinancialData.csv")